# Lesson 3 Demo 3: Focus on Clustering Column

In [1]:
# Import Apache Cassandra

import cassandra


## Let's create a connection to the databasse


In [2]:
from cassandra.cluster import Cluster

try:
    cluster = Cluster(['127.0.0.1']) # If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

### Let's create a keyspace to do our work in


In [3]:
try:
    session.execute("""
    create keyspace if not exists udacity
    with replication =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}"""
                   )
except Exception as e:
    print(e)

### Connect to the keyspace. Compare this to how we had create a new session in PostgreSQL.

In [4]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


##### Our query  In this case is I would like to be able to get every album that was released by an Artist with Album Name in DESC Order and City in DESC order
```bash
select * from music_library WHERE ARTIST_NAME="The Beatles"
```

###### How should we model this data? What should be our Primary key and Partition Key? Since our data is looking for the __artist_name__ let's start with that. From there we will need to add other elements to make sure the key is unique. We also need to add city and album_name as clustering columns to the data. That should be enough to make the row key unique

In [5]:
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(year int, artist_name text, album_name text, city text, PRIMARY KEY (artist_name, album_name, city))"

try:
    session.execute(query)
except Exception as e:
    print(e)


### Let's insert some data into both tables



In [6]:
query = "INSERT INTO music_library (year, artist_name, album_name, city)"
query = query + "VALUES (%s, %s, %s, %s)"

try:
    session.execute(query, (1965, "The Beatles", "Rubber Soul", "Oxford"))
except Exception as e:
    print(e)
    

try:
    session.execute(query, (1970, "The Beatles", "Let it Be", "Liverpool"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1965, "The Who", "My Generation", "London"))
except Exception as e:
    print(e)

    
try:
    session.execute(query, (1966, "The Monkees", "The Monkees", "Los Angeles"))
except Exception as e:
    print(e)
    
    
try:
    session.execute(query, (1970, "The Carpenters", "Close To You", "San Diego"))
except Exception as e:
    print(e)
    
try:
    session.execute(query, (1964, "The Beatles", "Beatles For Sale", "London"))
except Exception as e:
    print(e)


### Validate if the data was inserted 



In [7]:
query = "SELECT * from music_library where artist_name='The Beatles'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:  # the for loop is for printing, it will not be required if executing in cqlsh
    print(row.artist_name, row.album_name, row.city,  row.year)

The Beatles Beatles For Sale London 1964
The Beatles Let it Be Liverpool 1970
The Beatles Rubber Soul Oxford 1965


### For the sake of the demo, let's drop the table

In [8]:
query = "drop table music_library"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    


### And Finally close the session and cluster connection

In [9]:
session.shutdown()
cluster.shutdown()